# Amazon SageMaker - Keras example
ここでは[Amazon SageMaker で簡単に Keras を使う方法](https://aws.amazon.com/jp/blogs/news/amazon-sagemaker-keras)のサンプルスクリプトを紹介します。カーネルは`conda_tensorflow_p36`を選択して下さい。

## データセットの準備
`keras`からmnistのデータセットをロードし、S3へアップロードします。

In [ ]:
import os

import keras
from keras.datasets import mnist
import numpy as np
import sagemaker


(x_train, y_train), (x_test, y_test) = mnist.load_data()

os.makedirs("./data", exist_ok = True)

np.savez('./data/train', image=x_train, label=y_train)
np.savez('./data/test', image=x_test, label=y_test)


sagemaker_session = sagemaker.Session()
bucket_name = sagemaker_session.default_bucket()
input_data = sagemaker_session.upload_data(path='./data', bucket=bucket_name, key_prefix='dataset/mnist')
print('Training data is uploaded to: {}'.format(input_data))

学習用スクリプトを`entry_point`に指定し、各種パラメータを指定して TensorFlow Estimator を初期化します。`py_version='py3'`、`script_mode=True`を指定しておきます。

## トレーニングジョブの作成

In [ ]:
from sagemaker.tensorflow import TensorFlow
from sagemaker import get_execution_role

role = get_execution_role()
estimator = TensorFlow(
    entry_point = "./mnist_cnn_sagemaker_tensorflow.py",
    role=role,
    train_instance_count=1,
    train_instance_type="ml.m5.xlarge",
    framework_version="1.12.0",
    py_version='py3',
    script_mode=True,
    hyperparameters={'batch-size': 64,
                     'num-classes': 10,
                     'epochs': 1})

estimator.fit(input_data)

## リアルタイム推論の実施

In [ ]:
predictor = estimator.deploy(instance_type='ml.m5.xlarge', initial_instance_count=1)

In [ ]:
%matplotlib inline
import random
import matplotlib.pyplot as plt

num_samples = 5
indices = random.sample(range(x_test.shape[0] - 1), num_samples)
images, labels = x_test[indices]/255, y_test[indices]

for i in range(num_samples):
    plt.subplot(1,num_samples,i+1)
    plt.imshow(images[i].reshape(28, 28), cmap='gray')
    plt.title(labels[i])
    plt.axis('off')
    
prediction = predictor.predict(images.reshape(num_samples, 28, 28, 1))['predictions']
prediction = np.array(prediction)
predicted_label = prediction.argmax(axis=1)
print('The predicted labels are: {}'.format(predicted_label))

## エンドポイントの削除

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint)